&Lstrok;adowanie potrzebnych bibliotek

In [ ]:
from bs4 import BeautifulSoup
from requests import get
import sqlite3
import re
import time

Definiowanie funkcji, kt&oacute;ra przejdzie przez wszystkie og&lstrok;oszenia (podstrony), pobierze odpowiednie dane i wstawi do kolumn w bazie danych

In [ ]:
def parse_page(number):
    page = get(f'{url}?pet_page={number}&pet_species=1&pet_weight=0&pet_age=0')
    bs = BeautifulSoup(page.content, 'html.parser')
    #petla przechodzaca przez wszystkie podstrony z ogloszeniami
    for pet in bs.find_all('div', class_='lower-content'):
        top_line = pet.find('h3', class_='pets-list-pet-name')
        #wyciaganie numeru
        pet_number = top_line.find('small').get_text().strip()
        #wyciaganie linka do ogloszenia
        link = top_line.find('a')
        link2 = link['href']
        subpage = get(f'https://napaluchu.waw.pl{link2}')
        #wejscie na podstrone
        bs_sub = BeautifulSoup(subpage.content, 'html.parser')
        body = bs_sub.find('section', class_='pets-container')
        title = body.find('div', class_='auto-container')
        #wyciaganie imienia
        pet_name = title.find('h2').get_text().strip().split()
        str_name = re.compile(r'[A-Za-z]+')
        if str_name.match(pet_name[0]):
            pet_name2 = pet_name[0]
        else:
            pet_name2 = 'brak'
        details = body.find('ul', class_='petdetails')
        #wyciaganie informacji o rasie
        str_breed = re.compile(r'W typie rasy: [\w]+[ ]?[\w]*')
        breed0 = str_breed.findall(details.find('li').get_text().strip())
        if len(breed0) == 0:
            breed2 = 'brak'
        else:
            breed = ' '.join(breed0).split(':')
            breed2 = breed[1].strip()
        #wyciaganie informacji o wieku
        str_age = re.compile(r'Wiek: [0-9]+[ ]?[\w]*')
        age0 =  str_age.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(age0) == 0:
            age2 = 'brak'
        else:
            age = ' '.join(age0).split(':')
            age2 = age[1].strip()
        #wyciaganie informacji o plci
        str_sex = re.compile(r'P\we\w: [\w]+')
        sex0 = str_sex.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(sex0) == 0:
            sex2 = 'brak'
        else:
            sex = ' '.join(sex0).split(':')
            sex2 = sex[1].strip()
        #wyciaganie informacji o wadze
        str_weight = re.compile(r'Waga: [\w ]+')
        weight0 = str_weight.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(weight0) == 0:
            weight2 = 'brak'
        else:
            weight = ' '.join(weight0).split(':')
            weight2 = weight[1].strip()
        #wyciaganie informacji o statusie
        str_status = re.compile(r'Status: [\w ]+')
        status0 = str_status.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(status0) == 0:
            status2 = 'brak'
        else:
            status = ' '.join(status0).split(':')
            status2 = status[1].strip()
        #wyciaganie informacji o dacie przyjecia
        str_admission = re.compile(r'Przyj\wty: [\w-]+')
        admission0 = str_admission.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(admission0) == 0:
            admission2 = 'brak'
        else:
            admission = ' '.join(admission0).split(':')
            admission2 = admission[1].strip()
        #wyciaganie informacji o dacie wydania
        str_release = re.compile(r'Wydany: [\w-]+')
        release0 = str_release.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(release0) == 0:
            release2 = 'brak'
        else:
            release = ' '.join(release0).split(':')
            release2 = release[1].strip()
        #wyciaganie informacji o miejscu znalezienia
        str_found = re.compile(r'Znaleziony: [\w .,]+')
        found0 = str_found.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(found0) == 0:
            found2 = 'brak'
        else:
            found = ' '.join(found0).split(':')
            found2 = found[1].strip()
        #wyciaganie informacji o boksie
        str_kennel = re.compile(r'Boks: [\w .,()]+')
        kennel0 = str_kennel.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(kennel0) == 0:
            kennel2 = 'brak'
        else:
            kennel = ' '.join(kennel0).split(':')
            kennel2 = kennel[1].strip()
        #wyciaganie informacji o grupie
        str_group_ = re.compile(r'Grupa: [\w .,()-]+')
        group_0 = str_group_.findall(body.find('ul', class_='petdetails').get_text().strip())
        if len(group_0) ==0:
            group_2 = 'brak'
        else:
            group_ = ' '.join(group_0).split(':')
            group_2 = group_[1].strip()
        #wrzucanie zmiennych do bazy danych
        cur.execute('INSERT INTO pets VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', 
                    (pet_number, pet_name2, breed2, age2, sex2, weight2, status2, admission2, release2, found2, kennel2, group_2))
        time.sleep(2)
    con.commit()

Definiowanie adresu url, po&lstrok;&aogon;czenie z nowo utworzon&aogon; baz&aogon;, tworzenie tabeli w bazie

In [ ]:
url = 'https://napaluchu.waw.pl/zwierzeta/znalazly-dom/'
con = sqlite3.connect('na_paluchu.db')
cur = con.cursor()

cur.execute('DROP TABLE IF EXISTS pets')
cur.execute('''CREATE TABLE pets
                (numer text, imie text, w_typie_rasy text, wiek text, plec text, waga text, status text, data_przyjecia text, data_wydania text, miejsce_znalezienia text, boks text, grupa text)''')
quit()

Tworzenie p&eogon;tli przechodz&aogon;cej przez wszystkie numery stron i zamykanie po&lstrok;&aogon;czenia z baz&aogon; danych.

In [ ]:
for number in range(1, 1335):
    time.sleep(1)
    parse_page(number)        

con.close()